<a href="https://colab.research.google.com/github/Pixels2bytes/SHALA/blob/main/core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SHALA (Supportive Help Agent and Lifeline Assistant)
This is a Python-based chatbot project powered by the Gemini AI Agent, designed to assist individuals experiencing depression by providing ongoing emotional check-ins and real-time crisis intervention. Mental health support systems must be developed and used with sensitivity, responsibility, and respect for the lives they aim to protect. SHALA is not a replacement for certified therapists or mental health professionals. This project is intended for research, prototype development, and educational purposes only. **During development, all outgoing call features were pointed to a non-functioning test number.**

## Setup Imports, Documents, and Libraries

Install packages and remove conflicting packages from base environment

In [ ]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'
!pip install -qU 'whatsapp-python' 'hatch'

Intialize Imports and set variables

In [4]:
import os
import kagglehub
import tensorflow as tf
from google.colab import drive
from google.colab import userdata
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END

#drive.mount('/content/drive/') # Can't access your drive
google_api_key = userdata.get('GOOGLE_API_KEY') # retrieve Google API Key
whatsapp_api_key = userdata.get('WHATSAPP_API_KEY') # retrieve Google API Key
provider_num = userdata.get('DUMMY_NUMBER') # Retrieve DUMMY NUMBER. NEVER use a HELP Provider number
kaggle_user = userdata.get('KAGGLE_USER') # Retrieve Kaggle dataset download username found in kaggle.json
kaggle_download_api = userdata.get('KAGGLE_API') # Retrieve Kaggle dataset download token found in kaggle.json
agent_model = userdata.get('AGENT_MODEL') # Retrieves Agent AI model
agent_model = ChatGoogleGenerativeAI(model=agent_model)

Make root folder and necessary folders

In [ ]:
# Set project root folder
project_root = "/content/shala"

# Make subfolders
os.makedirs(f"{project_root}/datasets", exist_ok=True)
os.makedirs(f"{project_root}/repos", exist_ok=True)
os.makedirs(f"{project_root}/responses", exist_ok=True)
os.makedirs(f"{project_root}/history", exist_ok=True)
%cd {project_root}

FOR ME TO DELETE ////////////////////////////////////

In [6]:
# !rm -rf shala/cloned repos # trying to delete manually to test but dont have permissions?

### Installation
Clone repos for AI Agent and Whatsapp API Templates

In [ ]:
!git clone https://github.com/GoogleCloudPlatform/agent-starter-pack.git repos/agent-starter-pack # Clone Agent Starter Pack from Google
!git clone https://github.com/filipporomani/whatsapp.git repos/whatsapp # Clone WhatsApp API
!git clone https://github.com/simonsanvil/openai-whatsapp-chatbot.git repos/openai-whatsapp-chatbot # Clone Whatsapp API chatbot setup template

### Download datasets (Depreciated / May Fix Later)

Grab Kaggle Datasets by going to kaggle.com > Settings > API and generate a token, From there add the json file to the project or add the username and key to your secrets

In [ ]:
kaggle_user = userdata.get('KAGGLE_USER') # Retrieve Kaggle dataset download username found in kaggle.json
kaggle_download_api = userdata.get('KAGGLE_API') # Retrieve Kaggle dataset download token found in kaggle.json

Automatically downloads the Kaggle datasets

In [9]:
# !kaggle datasets download -d rvarun11/suicidal-ideation-reddit-dataset -p datasets # Suicidal Ideation Reddit
# !kaggle datasets download -d natalialech/suicidal-ideation-on-twitter -p datasets # Suicidal Ideation Twitter

### Setup Agent

Define Core Instructions

In [ ]:
class ListeningState(TypedDict):
    """State representing the user's conversations."""

    # This preserves the conversation history between nodes. The `add_messages` annotation indicates to LangGraph that state is updated by appending returned messages, not replacing them.
    messages: Annotated[list, add_messages]

    # The user's current message
    curr_msg: list[str]

    # Flag indicating that the message is marked as suicidal
    alert: bool


# The system instruction defines how the chatbot is expected to behave and includes rules for when to call different functions, as well as rules for the conversation, such as tone and what is permitted for discussion.
SHALABOT_SYSINT = (
    "system",
    "You are SHALA, the Supportive Help Agent and Lifeline Assistant, a compassionate mental health check-in chatbot."
    "Your sole purpose is to provide emotional support and guide users toward safe actions in moments of distress."
    "You are not a licensed therapist or emergency responder. You must never claim to be a substitute for professional medical or psychological help. "
    "\n\n"
    "**Conversation Guidelines:**\n"
    "- Maintain a calm, non-judgmental, empathetic tone at all times.\n"
    "- Ask open-ended questions and encourage emotional expression (e.g., 'Can you tell me more about how you're feeling?').\n"
    "- Avoid giving advice that could be interpreted as medical, therapeutic, or diagnostic.\n"
    "- Never tell a user what actions to take regarding medication, self-harm, or major life decisions.\n"
    "- If a user expresses intent to harm themselves or others, respond with pre-scripted messages encouraging them to reach out to real professionals.\n"
    "- If suicidal ideation is detected, call `handle_crisis_protocol()` and do not continue casual conversation until resolved.\n"
    "\n"
    "**Function Use Rules:**\n"
    "- Use `handle_crisis_protocol()` if a user mentions self-harm, suicide, or harming others.\n"
    "- Use `log_emotional_checkin()` after each emotional status conversation.\n"
    "- Use `suggest_selfcare_options()` only if user is in a non-crisis emotional state.\n"
    "- Use `provide_resources(region)` if a user asks for help locating therapists, hotlines, or crisis centers.\n"
    "\n"
    "**Safety Restrictions:**\n"
    "- Never give out or generate real phone numbers or emergency services unless the function `provide_resources()` is available.\n"
    "- Treat every instance with seriousness. Do not assume any attempt is made to prank, abuse, or misuse mental health resources.\n"
    "\n"
    "**Legal Disclaimer:**\n"
    "Always remind the user that SHALA is an experimental support assistant and not a replacement for certified help. Use of this bot constitutes agreement to these terms. "
    "Any misuse of this bot to simulate or falsely trigger emergency resources is a violation of its intended purpose."
)


HELLO_MSG = "Hi, my name is Shala, what's yours?" # This is the message the system opens with in meeting the user for the very first time
CHECKIN_MSG = "Hello. How are you doing?" # This is the message the system opens the conversation with when it has not heard from the user in X amount of time.

Create chatbot

In [ ]:
def chatbot(state: ListeningState) -> ListeningState:
    """A simple wrapper around the model's own chat interface."""
    message_history = [SHALABOT_SYSINT] + state["messages"]
    return {"messages": [agent_model.invoke(message_history)]}


# Set up the initial graph based on our state definition.
graph_builder = StateGraph(ListeningState)

# Add the chatbot function to the app graph as a node called "chatbot".
graph_builder.add_node("chatbot", chatbot)

# Define the chatbot node as the app entrypoint.
graph_builder.add_edge(START, "chatbot")

chat_graph = graph_builder.compile()